Current Goal:
Ensure that data is being prepped in the most appropriate way possible

In [1]:
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Wrangling
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import env
from os import path 
import math
import os
from sklearn.preprocessing import StandardScaler, QuantileTransformer, PowerTransformer, RobustScaler, MinMaxScaler

# Statistical Tests
import scipy.stats as stats

# Visualizing
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns

In [2]:
def get_connection(database):
    '''
    Database: string; name of database that the url is being created for
    '''
    return f'mysql+pymysql://{env.user}:{env.password}@{env.host}/{database}'

In [3]:
sql_query = '''
SELECT prop.*, logerror, transactiondate, airconditioningdesc, architecturalstyledesc, buildingclassdesc, heatingorsystemdesc, propertylandusedesc, storydesc, typeconstructiondesc
FROM properties_2017 as prop 
INNER JOIN (
    SELECT id, p.parcelid, logerror, transactiondate
    FROM predictions_2017 AS p
    INNER JOIN (
        SELECT parcelid,  MAX(transactiondate) AS max_date
        FROM predictions_2017 
        GROUP BY (parcelid)) AS sub
            ON p.parcelid = sub.parcelid
        WHERE p.transactiondate = sub.max_date
        ) AS subq
    ON prop.parcelid = subq.parcelid
LEFT JOIN airconditioningtype
    ON prop.airconditioningtypeid = airconditioningtype.airconditioningtypeid
LEFT JOIN architecturalstyletype
    ON prop.architecturalstyletypeid = architecturalstyletype.architecturalstyletypeid
LEFT JOIN buildingclasstype 
    ON prop.buildingclasstypeid = buildingclasstype.buildingclasstypeid
LEFT JOIN heatingorsystemtype
    ON prop.heatingorsystemtypeid = heatingorsystemtype.heatingorsystemtypeid
LEFT JOIN propertylandusetype
    ON prop.propertylandusetypeid = propertylandusetype.propertylandusetypeid
LEFT JOIN storytype
    ON prop.storytypeid = storytype.storytypeid
LEFT JOIN typeconstructiontype
    ON prop.typeconstructiontypeid = typeconstructiontype.typeconstructiontypeid
WHERE latitude IS NOT NULL 
    AND longitude IS NOT NULL
    AND transactiondate BETWEEN '2017-01-01' AND '2017-12-31';
'''

In [4]:
def acquire_cache_zillow():
    if not path.isfile('zillow.csv'):
        url = get_connection('zillow')
        zillow = pd.read_sql(sql_query, url)
        zillow.to_csv('zillow.csv')
    zillow = pd.read_csv('zillow.csv')
    zillow.drop(columns=['Unnamed: 0'], inplace=True)
    return zillow

In [5]:
def handle_missing_values(df, prop_required_column = .5, prop_required_row = .75):
    threshold = int(round(prop_required_column*len(df.index),0))
    df.dropna(axis=1, thresh=threshold, inplace=True)
    threshold = int(round(prop_required_row*len(df.columns),0))
    df.dropna(axis=0, thresh=threshold, inplace=True)
    return df

In [6]:
def map_county(row):
    if row['fips'] == 6037:
        return 'Los Angeles'
    elif row['fips'] == 6059:
        return 'Orange'
    elif row['fips'] == 6111:
        return 'Ventura'

In [8]:
def wrangle_zillow(df):
    df = df[(df.propertylandusedesc == 'Single Family Residential')|(df.propertylandusedesc == 'Condominium')|(df.propertylandusedesc == 'Planned Unit Development')|(df.propertylandusedesc == 'Mobile Home')|(df.propertylandusedesc == 'Manufactured, Modular, Prefabricated Homes')|(df.propertylandusedesc == 'Residential General')]
    df = df[df.bedroomcnt > 0]
    df = df[~(df.unitcnt > 1)]
    handle_missing_values(df)
    df.drop(columns=['assessmentyear', 'calculatedbathnbr', 'finishedsquarefeet12', 'propertyzoningdesc', 'regionidcity', 'roomcnt', 'unitcnt'], inplace=True)
    df = df.dropna(subset=['censustractandblock'])
    df = df.dropna(subset=['regionidzip'])
    columns_to_change = ['id', 'parcelid', 'fips', 'heatingorsystemtypeid', 'propertylandusetypeid', 'rawcensustractandblock', 'regionidcounty', 'regionidzip', 'censustractandblock']
    for column in columns_to_change:
        df[column] = df[column].astype(object)
    df['fips'] = df.apply(lambda row: map_county(row), axis = 1)
    return df